In [2]:
import pandas as pd
import numpy as np
import re, os, math, sklearn, datetime, pickle

In [113]:
df_movies = pd.read_csv('movies.csv')
df_ratings = pd.read_csv('ratings.csv')
df_tags = pd.read_csv('tags.csv')

In [96]:
df_movies.head(10)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


In [97]:
df_ratings.head(10)

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
5,1,70,3.0,964982400
6,1,101,5.0,964980868
7,1,110,4.0,964982176
8,1,151,5.0,964984041
9,1,157,5.0,964984100


### 1. Filter movies with 5 or more ratings

#### Create a dataframe (df_movie_ratings_count_min5) that stores each movie that has more than 4 ratings along with the number of its ratings

Group the ratings_df by the movieId and count the rows with the same movieId

In [100]:
df_movie_ratings_count = ratings_df.groupby('movieId').count()

Keep only the index and the first column and rename the first column for beautifying purposes

In [105]:
df_movie_ratings_count = df_movie_ratings_count.iloc[:,:1]

df_movie_ratings_count.rename(columns = {'userId':'count'}, inplace = True)

In [106]:
df_movie_ratings_count.head(33)

,count
movieId,
1,215
2,110
3,52
4,7
5,49
6,102
7,54
8,8
9,16


In [109]:
df_movie_ratings_count_min5= df_movie_ratings_count.loc[df_movie_ratings_count['count'] > 4]

In [110]:
df_movie_ratings_count_min5.head(40)

,count
movieId,
1,215
2,110
3,52
4,7
5,49
6,102
7,54
8,8
9,16


In [177]:
print(len(df_movie_ratings_count_min5.index))
print(len(df_movie_ratings_count.index))
print(len(df_movie_ratings_count.index)-len(df_movie_ratings_count_min5.index), "movies have been removed")

3650
9724
6074 movies have been removed


#### Create df_ratings_min5 and copy only the movies from df_ratings whose id belongs in df_movie_ratings_count_min5

In [139]:
df_ratings_min5 = df_ratings.loc[df_ratings['movieId'].isin(df_movie_ratings_count_min5.index)]

In [169]:
print(df_ratings_min5.shape)
print(df_ratings.shape)

(90274, 4)
(100836, 4)


#### Check that movies with fewer than 5 ratings have indeed been removed in df_ratings_min5

In [164]:
df_movies.loc[df_movies['movieId']==40]

,movieId,title,genres
36,40,"Cry, the Beloved Country (1995)",Drama


In [162]:
df_ratings.loc[df_ratings['movieId']==40]

,userId,movieId,rating,timestamp
47991,311,40,3.5,1057854804
84607,544,40,5.0,850688776


In [165]:
df_ratings_min5.loc[df_ratings_min5['movieId']==40]

,userId,movieId,rating,timestamp


### 2. Split data 90/10

In [203]:
X_train = df_ratings_min5.sample(frac=0.9).reset_index(drop=True)
X_test = df_ratings.sample(frac=0.1).reset_index(drop=True)

In [204]:
print(X_train.shape)
print(X_test.shape)

(81247, 4)
(10084, 4)


#### Φτιάχνω ένα array με όλες τις διαφορετικές ταινίες

In [6]:
movies_list = list(set(ratings_df.movieId))

In [7]:
len(movies_list)

9724

In [15]:
movies_listss = list((ratings_df.movieId))

In [18]:
movies_listss

[1,
 3,
 6,
 47,
 50,
 70,
 101,
 110,
 151,
 157,
 163,
 216,
 223,
 231,
 235,
 260,
 296,
 316,
 333,
 349,
 356,
 362,
 367,
 423,
 441,
 457,
 480,
 500,
 527,
 543,
 552,
 553,
 590,
 592,
 593,
 596,
 608,
 648,
 661,
 673,
 733,
 736,
 780,
 804,
 919,
 923,
 940,
 943,
 954,
 1009,
 1023,
 1024,
 1025,
 1029,
 1030,
 1031,
 1032,
 1042,
 1049,
 1060,
 1073,
 1080,
 1089,
 1090,
 1092,
 1097,
 1127,
 1136,
 1196,
 1197,
 1198,
 1206,
 1208,
 1210,
 1213,
 1214,
 1219,
 1220,
 1222,
 1224,
 1226,
 1240,
 1256,
 1258,
 1265,
 1270,
 1275,
 1278,
 1282,
 1291,
 1298,
 1348,
 1377,
 1396,
 1408,
 1445,
 1473,
 1500,
 1517,
 1552,
 1573,
 1580,
 1587,
 1617,
 1620,
 1625,
 1644,
 1676,
 1732,
 1777,
 1793,
 1804,
 1805,
 1920,
 1927,
 1954,
 1967,
 2000,
 2005,
 2012,
 2018,
 2028,
 2033,
 2046,
 2048,
 2054,
 2058,
 2078,
 2090,
 2093,
 2094,
 2096,
 2099,
 2105,
 2115,
 2116,
 2137,
 2139,
 2141,
 2143,
 2161,
 2174,
 2193,
 2253,
 2268,
 2273,
 2291,
 2329,
 2338,
 2353,
 2366,
 